In [2]:
import pandas as pd
import numpy as np
import random

In [68]:
n_splits=10
target_col = "HeartDiseaseorAttack"
missing_sample_size = 0.15

In [69]:
# Read folds that are available
# Create empty lists to store train and test DataFrames
train_datasets = []
test_datasets = []

for fold_num in range(1, n_splits+1):
    train_file_path = f"foldy/fold_{fold_num}_train.csv"
    test_file_path = f"foldy/fold_{fold_num}_test.csv"
    
    # Load the train and test fold data into DataFrames
    train_fold = pd.read_csv(train_file_path)
    test_fold = pd.read_csv(test_file_path)
    
    train_datasets.append(train_fold)
    test_datasets.append(test_fold)
print("Folds data were loaded successfully!")

Folds data were loaded successfully!


In [70]:
for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):
    print(f'Fold {fold_num} train: {train_data.shape}')
    print(f'Fold {fold_num} test: {test_data.shape}')


Fold 1 train: (206803, 22)
Fold 1 test: (22978, 22)
Fold 2 train: (206803, 22)
Fold 2 test: (22978, 22)
Fold 3 train: (206803, 22)
Fold 3 test: (22978, 22)
Fold 4 train: (206803, 22)
Fold 4 test: (22978, 22)
Fold 5 train: (206803, 22)
Fold 5 test: (22978, 22)
Fold 6 train: (206803, 22)
Fold 6 test: (22978, 22)
Fold 7 train: (206803, 22)
Fold 7 test: (22978, 22)
Fold 8 train: (206803, 22)
Fold 8 test: (22978, 22)
Fold 9 train: (206803, 22)
Fold 9 test: (22978, 22)
Fold 10 train: (206803, 22)
Fold 10 test: (22978, 22)


In [71]:
records_to_nan_matrix = pd.DataFrame(1, columns=train_data.columns, index=range(train_data.shape[0]))
records_to_nan_matrix.head()

,HeartDiseaseorAttack,HighBP,HighChol,CholCheck,BMI,Smoker,Stroke,Diabetes,PhysActivity,Fruits,...,AnyHealthcare,NoDocbcCost,GenHlth,MentHlth,PhysHlth,DiffWalk,Sex,Age,Education,Income
0,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
3,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
4,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1


In [72]:
unique_numbers = random.sample(range(0, train_data.shape[0] + 1), int(missing_sample_size*train_data.shape[0]))
print(f'Number of randomly selected observations: {len(set(unique_numbers))}')

Number of randomly selected observations: 31020


In [73]:
features = list(train_data.columns)
features.remove(target_col)

In [74]:
for index_to_nan in unique_numbers:
    sel_columns = random.choice([1,2])
    columns_to_add_nans = random.sample(features, sel_columns)

    for col_to_nan in columns_to_add_nans:
        records_to_nan_matrix.at[index_to_nan, col_to_nan] = np.NaN

In [75]:
records_to_nan_matrix.to_csv('foldy/records_to_nan_matrix_15pct.csv', index=False)

In [76]:
# Create a directory for the folds if it doesn't exist
import os
if not os.path.exists('missing_values_folds_15pct'):
    os.makedirs('missing_values_folds_15pct')

In [77]:
for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):
    train_data_with_nans = train_data * records_to_nan_matrix

    # Save each fold as a CSV file
    train_data_with_nans.to_csv(f'missing_values_folds_15pct/fold_{fold_num}_train.csv', index=False)
    test_data.to_csv(f'missing_values_folds_15pct/fold_{fold_num}_test.csv', index=False)

In [ ]:
# for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

#     if fold_num == 1:
#         patients_to_nan = train_data.sample(frac=missing_sample_size, random_state=42, ignore_index=True)
#         features = list(train_data.columns)
#         features.remove(target_col)

# patients_to_nan.to_csv(f'folds/naned/patients_to_nan_train.csv', index=False)


# patient_nan_features_dict = {}
# for patient_idx, patient_row in patients_to_nan.iterrows():
#     # select how many features and which ones will be NaNed out for a patient idx
#     sel_columns = random.choice([1,2])
#     columns_to_add_nans = random.sample(features, sel_columns)
#     patient_nan_features_dict[patient_idx] = columns_to_add_nans    

In [ ]:
# patient_nan_dict = patients_to_nan.to_dict(orient='index')

In [ ]:
# # for every fold find that patient in train data and if exists - NaN out some features
# for fold_num, (train_data, test_data) in enumerate(zip(train_datasets, test_datasets),1):

#     for ft_idx, ft_row in train_data.iterrows():

#         for patient_idx, patient_row in patient_nan_dict.items():

#             if patient_row == ft_row.to_dict():
#                 # print(f'Found a patient in train set in fold {fold_num} -> imputing NaN in {columns_to_add_nans} columns')        
#                 for col in patient_nan_features_dict[patient_idx]:
#                     train_data.at[ft_idx, col] = np.NaN
#             # patients are unique (dataframe has no duplicates) - so once one patient is found in one folds' training set, the search in that fold can be finished
#             # break
#     print(f'Fold {fold_num} finished successfully')
#     # Save each fold as a CSV file
#     train_data.to_csv(f'folds/naned/fold_{fold_num}_train.csv', index=False)
#     test_data.to_csv(f'folds/naned/fold_{fold_num}_test.csv', index=False)

